In [3]:
pip install keras-core

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 7.4 MB/s eta 0:00:00


In [44]:
# install transformers with sentencepiece
!pip install transformers[sentencepiece]

In [45]:
file = open("transcript.txt", "r")
FileContent = file.read().strip()

In [46]:
# display file content
FileContent

"Yeah. Yeah, sure. It kinda does make sense, doesn't it, because when we get into the end of meeting we're kind of talking about action and design as opposed to background. Everything I have is kinda background. Mm-hmm. Uh that sounds. Sure. Okay. Sure. Yeah, cool. Why don't I get that? Hmm. Okay. Okay. Um alright so c is it function F_ eight? Hmm. Come on. I think it's working. Okay great s so let me just start this. Okay great. So um uh s move on. Uh-huh oh where'd it all go? It's not good. Okay lemme just see where I can find it. This looks more like it. I think I just opened up the template. Sorry about that. Okay alright so let's have a look here. Okay so this was the method that um I've taken. Uh basically what I wanna do here, before we get into it uh too far, is I want to show you all the background information I have that I think we need to acknowledge if we want this to be successful. And uh and then sorta g go through some of the way that I've dealt with that information, an

In [47]:
# total characters in the file
len(FileContent)


39722

In [48]:
# import and initialize the tokenizer and model from the checkpoint
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

checkpoint = "sshleifer/distilbart-cnn-12-6"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

In [49]:

# max tokens including the special tokens
tokenizer.model_max_length

1024

In [50]:
# max tokens excluding the special tokens
tokenizer.max_len_single_sentence

1022

In [51]:
# number of special tokens
tokenizer.num_special_tokens_to_add()


2

In [52]:
# extract the sentences from the document
import nltk
nltk.download('punkt')
sentences = nltk.tokenize.sent_tokenize(FileContent)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [53]:
# find the max tokens in the longest sentence
max([len(tokenizer.tokenize(sentence)) for sentence in sentences])


93

In [54]:
# initialize
length = 0
chunk = ""
chunks = []
count = -1
for sentence in sentences:
  count += 1
  combined_length = len(tokenizer.tokenize(sentence)) + length # add the no. of sentence tokens to the length counter

  if combined_length  <= tokenizer.max_len_single_sentence: # if it doesn't exceed
    chunk += sentence + " " # add the sentence to the chunk
    length = combined_length # update the length counter

    # if it is the last sentence
    if count == len(sentences) - 1:
      chunks.append(chunk.strip()) # save the chunk

  else:
    chunks.append(chunk.strip()) # save the chunk

    # reset
    length = 0
    chunk = ""

    # take care of the overflow sentence
    chunk += sentence + " "
    length = len(tokenizer.tokenize(sentence))
len(chunks)

10

In [55]:
[len(tokenizer.tokenize(c)) for c in chunks]

[1014, 984, 960, 1005, 1016, 994, 991, 1005, 1017, 576]

In [56]:
[len(tokenizer(c).input_ids) for c in chunks]

[1016, 986, 962, 1007, 1018, 996, 993, 1007, 1019, 578]

In [57]:
sum([len(tokenizer(c).input_ids) for c in chunks])

9582

In [58]:
len(tokenizer(FileContent).input_ids)

Token indices sequence length is longer than the specified maximum sequence length for this model (9564 > 1024). Running this sequence through the model will result in indexing errors


9564

In [59]:
sum([len(tokenizer.tokenize(c)) for c in chunks])

9562

In [60]:
len(tokenizer.tokenize(FileContent))

9562

In [61]:
# inputs to the model
inputs = [tokenizer(chunk, return_tensors="pt") for chunk in chunks]

In [62]:
for input in inputs:
  output = model.generate(**input)
  print(tokenizer.decode(*output, skip_special_tokens=True))

 Market research shows that TV remote control has a fancy look and feel, not a functional look or or feel, the number one thing that was found was that television remote control was not functional. Number two was that it be innovative without a adding unnecessary functional bits to it, and third priority is that it has to be user friendly while still having technology.
 Style is number one thing in the in the market of who we're selling to. Innovative design technology's also a must in that it's seen it'd be seen to be uh cutting edge, but ease of use t has to be insured throughout. And then at the end there are vibrant natural colours.
 We need to have something that unifies a lot of the different concepts, and if we think that what we are w our number one marketing motive is the look and feel. We are leaning quite a bit to the side of being low-tech, rubber buttons plastic frame, it's almost like we're reproducing the remote control that's out there. We're gonna need to put in a real